<a href="https://colab.research.google.com/github/Rai90/ggepy-tutorial/blob/main/gge_py_3_reduce_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [194]:
import ee

# Trigger the authentication flow.
ee.Authenticate()



True

In [195]:
import geemap
ee.Initialize(project='ee-raischwalbert')

# Load a featurecollection (vectorial file)


In [196]:
states = ee.FeatureCollection("WM/geoLab/geoBoundaries/600/ADM1")

states = states.filter(ee.Filter.Or(
    ee.Filter.eq("shapeName", "Kansas"),
    ee.Filter.eq("shapeName", "Colorado"),
    ee.Filter.eq("shapeName", "Oklahoma")))





# Plot vectorial data

In [197]:
types = states.toList(10).map(lambda x: ee.Feature(x).get("shapeName")).distinct().getInfo()


styled_fc = geemap.ee_vector_style(
    states, column="shapeName", labels=types, fillColor=["red","green","blue"])


Map = geemap.Map()
Map.addLayer(styled_fc)

Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

# Summarize over time

In [198]:
prism = ee.ImageCollection("OREGONSTATE/PRISM/AN81m") \
  .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
  .select('ppt') \
  .reduce(ee.Reducer.stdDev()) \
  .clip(states)

vis_params = {
    'min': 20,
    'max': 70,
    'palette': ['blue', 'white', 'red']
}

Map.addLayer(prism, vis_params)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

# Summarize over region

In [199]:
prism = ee.ImageCollection("OREGONSTATE/PRISM/AN81m") \
  .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
  .select('ppt').filterBounds(states)


#get the first image

prism = ee.Image(prism.first())

proj = prism.projection()
res = proj.nominalScale()
print("resolution = ", res.getInfo())
print("projection = ", proj.getInfo())

# display(prism)

stats = prism.reduceRegions(
    collection=states,
    reducer=ee.Reducer.median(),
    scale=res,
    crs=proj
)


# display(stats)




resolution =  4638.312116390109
projection =  {'type': 'Projection', 'wkt': 'GEOGCS["GCS_North_American_1983", \n  DATUM["North_American_Datum_1983", \n    SPHEROID["GRS_1980", 6378137.0, 298.257222101]], \n  PRIMEM["Greenwich", 0.0], \n  UNIT["degree", 0.017453292519943295], \n  AXIS["Longitude", EAST], \n  AXIS["Latitude", NORTH]]', 'transform': [0.0416666666667, 0, -125.02083333333336, 0, -0.0416666666667, 49.93749999999975]}


In [200]:
ppt = stats.toList(10).map(lambda x: ee.Feature(x).get("median")).distinct().getInfo()

ppt

styled_stats = geemap.ee_vector_style(
    stats, column="median", labels=ppt, fillColor=["red","green","blue"])


Map2 = geemap.Map()
Map2.addLayer(styled_stats)

Map2

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…